# Como um Agent é construído

A ideia central dos agentes é usar um modelo de linguagem para escolher uma sequência de ações a serem executadas. Em cadeias, uma sequência de ações é codificada diretamente (no código). Em agentes, um modelo de linguagem é usado como um motor de raciocínio para determinar quais ações devem ser tomadas e em qual ordem.

## Criando as tools que usaremos

In [1]:
import requests
import datetime

from langchain.agents import tool
from pydantic import BaseModel, Field #Importação atualizada

import wikipedia
wikipedia.set_lang('pt')

class RetornTempArgs(BaseModel):
    latitude: float = Field(description='Latitude da localidade que buscamos a temperatura')
    longitude: float = Field(description='Longitude da localidade que buscamos a temperatura')


@tool(args_schema=RetornTempArgs)
def retorna_temperatura_atual(latitude: float, longitude: float):
    '''Retorna a temperatura atual para uma dada coordenada'''

    URL = 'https://api.open-meteo.com/v1/forecast'

    params = {
        'latitude': latitude,
        'longitude': longitude,
        'hourly': 'temperature_2m',
        'forecast_days': 1,
    }

    resposta = requests.get(URL, params=params)
    if resposta.status_code == 200:
        resultado = resposta.json()
        
        hora_agora = datetime.datetime.now(datetime.UTC).replace(tzinfo=None)
        lista_horas = [datetime.datetime.fromisoformat(temp_str) for temp_str in resultado['hourly']['time']]
        index_mais_prox = min(range(len(lista_horas)), key=lambda x: abs(lista_horas[x] - hora_agora))

        temp_atual = resultado['hourly']['temperature_2m'][index_mais_prox]
        return f'{temp_atual}ºC'
    else:
        raise Exception(f'Request para API {URL} falhou: {resposta.status_code}')

@tool
def busca_wikipedia(query: str):
    """Faz busca no wikipedia e retorna resumos de páginas para a query"""
    titulos_paginas = wikipedia.search(query)
    resumos = []
    for titulo in titulos_paginas[:3]:
        try:
            wiki_page = wikipedia.page(title=titulo, auto_suggest=True)
            resumos.append(f'Título da página: {titulo}\nResumo: {wiki_page.summary}')
        except:
            pass
    if not resumos:
        return 'Busca não teve retorno'
    else:
        return '\n\n'.join(resumos)

## Revisando a utilização das tools

In [22]:
from langchain.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.utils.function_calling import convert_to_openai_function

from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv())

prompt = ChatPromptTemplate.from_messages([
    ('system', 'Você é um assistente amigável chamado Isaac'),
    ('user', '{input}')
])
chat = ChatOpenAI(model='gpt-4o-mini')

tools = [busca_wikipedia, retorna_temperatura_atual]
tools_json = [convert_to_openai_function(tool) for tool in tools]
tool_run = {tool.name: tool for tool in tools}

chain = prompt | chat.bind(functions=tools_json)

In [23]:
from langchain.agents.output_parsers import OpenAIFunctionsAgentOutputParser

chain = prompt | chat.bind(functions=tools_json) | OpenAIFunctionsAgentOutputParser()

In [24]:
resposta = chain.invoke({'input': 'Qual a temperatura em Bauru?'})
resposta

AgentActionMessageLog(tool='retorna_temperatura_atual', tool_input={'latitude': -23.0056, 'longitude': -49.0574}, log="\nInvoking: `retorna_temperatura_atual` with `{'latitude': -23.0056, 'longitude': -49.0574}`\n\n\n", message_log=[AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"latitude":-23.0056,"longitude":-49.0574}', 'name': 'retorna_temperatura_atual'}, 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 33, 'prompt_tokens': 122, 'total_tokens': 155, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_b8bc95a0ac', 'finish_reason': 'function_call', 'logprobs': None}, id='run-482ed94d-8472-45af-9740-ae0aa2d7060a-0', usage_metadata={'input_tokens': 122, 'output_tokens': 33, 'total_tokens': 155, 'input_token_details': {

In [25]:
resposta.tool

'retorna_temperatura_atual'

In [26]:
resposta.tool_input

{'latitude': -23.0056, 'longitude': -49.0574}

In [27]:
resposta.message_log

[AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"latitude":-23.0056,"longitude":-49.0574}', 'name': 'retorna_temperatura_atual'}, 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 33, 'prompt_tokens': 122, 'total_tokens': 155, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_b8bc95a0ac', 'finish_reason': 'function_call', 'logprobs': None}, id='run-482ed94d-8472-45af-9740-ae0aa2d7060a-0', usage_metadata={'input_tokens': 122, 'output_tokens': 33, 'total_tokens': 155, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})]

## Adicionando o racicínio do agent as mensagens (agent_scretchpad)

Temos que adicionar junto as nossas mensagens um campo que armazenará o raciocínio atual do modelo chamado agent_scratchpad. Para isso, utilizamos um MessagesPlaceholder ao nosso prompt. Ele guardará espaço para o raciocínio e, caso o modelo não esteja gerando um raciocínio no momento, o MessagesPlaceholder não é utilizado.

In [28]:
from langchain.prompts import MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages([
    ('system', 'Você é um assistente amigável chamado Isaac'),
    ('user', '{input}'),
    MessagesPlaceholder(variable_name='agent_scratchpad')
])

chain = prompt | chat.bind(functions=tools_json) | OpenAIFunctionsAgentOutputParser()

In [29]:
resposta_inicial = chain.invoke({
    'input': 'Qual a temperatura em Bauru?',
    'agent_scratchpad': []})
resposta_inicial

AgentActionMessageLog(tool='retorna_temperatura_atual', tool_input={'latitude': -23.3136, 'longitude': -49.0682}, log="\nInvoking: `retorna_temperatura_atual` with `{'latitude': -23.3136, 'longitude': -49.0682}`\n\n\n", message_log=[AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"latitude":-23.3136,"longitude":-49.0682}', 'name': 'retorna_temperatura_atual'}, 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 33, 'prompt_tokens': 122, 'total_tokens': 155, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_b8bc95a0ac', 'finish_reason': 'function_call', 'logprobs': None}, id='run-600e43fc-2ac8-4add-bd5f-5d5e1aad8021-0', usage_metadata={'input_tokens': 122, 'output_tokens': 33, 'total_tokens': 155, 'input_token_details': {

In [30]:
observacao = tool_run[resposta_inicial.tool].run(resposta_inicial.tool_input)
observacao

'18.4ºC'

Podemos utilizar a função format_to_openai_function_messages para modificar o formato da resposta de forma que ela possa ser enviada, junto da observação, de volta ao modelo. No caso o que está ocorrendo é o modelo está pedindo que uma tool seja rodada, estamos rodando a tool e gerando uma observação, e a gora enviamos novamente para o modelo a pergunta original, a mensagem do próprio modelo dizendo que precisava que a tool fosse rodada e a observação gerada pela ferramenta.

In [31]:
from langchain.agents.format_scratchpad import format_to_openai_function_messages

format_to_openai_function_messages([(resposta_inicial, observacao)])

[AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"latitude":-23.3136,"longitude":-49.0682}', 'name': 'retorna_temperatura_atual'}, 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 33, 'prompt_tokens': 122, 'total_tokens': 155, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_b8bc95a0ac', 'finish_reason': 'function_call', 'logprobs': None}, id='run-600e43fc-2ac8-4add-bd5f-5d5e1aad8021-0', usage_metadata={'input_tokens': 122, 'output_tokens': 33, 'total_tokens': 155, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}),
 FunctionMessage(content='18.4ºC', additional_kwargs={}, response_metadata={}, name='retorna_temperatura_atual')]

In [32]:
resposta_final = chain.invoke({
    'input': 'Qual a temperatura em Bauru?',
    'agent_scratchpad': format_to_openai_function_messages([(resposta_inicial, observacao)])})
resposta_final

AgentFinish(return_values={'output': 'A temperatura atual em Bauru é de 18,4ºC.'}, log='A temperatura atual em Bauru é de 18,4ºC.')

### Criando um loop de racicínio

Por fim, podemos criar um loop que adiciona a automaticamente as chamadas de função e observações e fica chamando o modelo novamente até que a mensagem de AgentFinish seja recebida.

In [33]:
from langchain_core.agents import AgentFinish #Importação do AgentFinish atualizada

def run_agent(input):
    passos_intermediarios = []
    while True:
        resposta = chain.invoke({
            'input': input,
            'agent_scratchpad': format_to_openai_function_messages(passos_intermediarios)
        })
        if isinstance(resposta, AgentFinish):
            return resposta
        observacao = tool_run[resposta.tool].run(resposta.tool_input)
        passos_intermediarios.append((resposta, observacao))


In [34]:
run_agent('Qual é a temperatura de Bauru?')

AgentFinish(return_values={'output': 'A temperatura atual em Bauru é de 18,2ºC.'}, log='A temperatura atual em Bauru é de 18,2ºC.')

Modificamos um pouco o formato para padronizar ao funcionamento do LangChain para agents

In [15]:
from langchain_core.agents import AgentFinish #Importação do AgentFinish atualizada
from langchain.schema.runnable import RunnablePassthrough


pass_through = RunnablePassthrough.assign(
    agent_scratchpad = lambda x: format_to_openai_function_messages(x['intermediate_steps'])
)
chain = pass_through | prompt | chat.bind(functions=tools_json) | OpenAIFunctionsAgentOutputParser()

def run_agent(input):
    passos_intermediarios = []
    while True:
        resposta = chain.invoke({
            'input': input,
            'intermediate_steps': passos_intermediarios
        })
        if isinstance(resposta, AgentFinish):
            return resposta
        observacao = tool_run[resposta.tool].run(resposta.tool_input)
        passos_intermediarios.append((resposta, observacao))


In [35]:
pass_through.invoke({'input': 'Qual é a temperatura de Bauru?', 'intermediate_steps': []})

{'input': 'Qual é a temperatura de Bauru?',
 'intermediate_steps': [],
 'agent_scratchpad': []}

In [36]:
run_agent('Qual é a temperatura de Bauru?')

AgentFinish(return_values={'output': 'A temperatura atual em Bauru é de 19.4ºC.'}, log='A temperatura atual em Bauru é de 19.4ºC.')

## O que temos no final?

### Um Agent

In [38]:
prompt = ChatPromptTemplate.from_messages([
    ('system', 'Você é um assistente amigável chamado Isaac'),
    ('user', '{input}'),
    MessagesPlaceholder(variable_name='agent_scratchpad')
])

pass_through = RunnablePassthrough.assign(
    agent_scratchpad = lambda x: format_to_openai_function_messages(x['intermediate_steps'])
)
chain = pass_through | prompt | chat.bind(functions=tools_json) | OpenAIFunctionsAgentOutputParser()

### Um AgentExecutor

In [39]:
def run_agent(input):
    passos_intermediarios = []
    while True:
        resposta = chain.invoke({
            'input': input,
            'intermediate_steps': passos_intermediarios
        })
        if isinstance(resposta, AgentFinish):
            return resposta
        observacao = tool_run[resposta.tool].run(resposta.tool_input)
        passos_intermediarios.append((resposta, observacao))

In [43]:
run_agent('Qual a temperatura agora na cidade de SP')

AgentFinish(return_values={'output': 'A temperatura atual na cidade de São Paulo é de 19,9ºC.'}, log='A temperatura atual na cidade de São Paulo é de 19,9ºC.')